In [1]:
import numpy as np
# 역전파
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [2]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input   # 입력 변수를 기억 ( 보관 )
        return output
        
def forward(self, x):
        raise NotImplementedError()

def backward(self, gy):
        raise NotImplementedError()

In [3]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [4]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [5]:
# y = (eˣ²)² 순전파 코드
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [6]:
# y의 x에 대한 미분 결과
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [7]:
import numpy as np
# 역전파 자동화
# 연산 그래프의 노드
class Variable:
    def __init__(self, data):
        self.data = data # 변수의 값 == data
        self.grad = None # 역전파 시 기울기 ( gradient )
        self.creator = None # 해당 변수를 생성한 함수 저장

    # 해당 변수를 생성한 함수 저장
    def set_creator(self, func):
        self.creator = func

In [8]:
# 그래프의 엣지 ( Edge ) == 노드와 오드 사이를 연결하는 선
class Function:
    # 입력 변수를 받아 순전파 수행, 순전파 결과를 새로운 Variable 객체로 감싸서 반환
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y) # 순전파를 수행할 때, 출력 Variable객체에 해당하는 함수를 생성자로 설정하는 부분
        output.set_creator(self)   # 출력 변수에 창조자 설정
        self.input = input   # 입력 변수를 기억 ( 보관 )
        self.output = output   # 출력도 저장
        return output

In [9]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [10]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프르이 노드들을 거꾸로 거슬러 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [11]:
# 역전파 순서 코드
y.grad = np.array(1.0)   # dy / dy라 무조건 1임

C = y.creator   # 1. 함수를 가져온다.
b = C.input   # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)   # 3. 함수의 backward 메서드를 호출한다

In [12]:
B = b.creator   # 1. 함수를 가져온다.
a = B.input   #2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad)   #3. 함수의 backward 메서드를 호출한다.

In [13]:
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [14]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator   # 1. 함수를 가져온다.
        if f is not None:
            x = f.input   # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)   # 3. 함수의 backward 메서드를 호출한다.
            x.backward()   # 하나 앞 변수의 backward 메서드를 호줄한다. ( 재귀 )

In [15]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [16]:
# Step 8. 재귀에서 반복문으로
# ‘재귀를 사용한 구현’을 ‘반복문을 이용한 구현’으로 수정

In [17]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()   # 함수를 가져온다.
            x, y = f.input, f.output   # 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)   # backward 메서드를 호출한다.
            
            if x.creator is not None:
                funcs.append(x.creator) # 얻은 입력 변수로 하나 앞의 함수를 리스트에 추가한다.

In [18]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [19]:
# Step 9. 함수를 더 편리하게

In [20]:
def square(x):
    # Square 클래스의 인스턴스를 생성합니다.
    f = Square()
    
    # 생성된 Square 클래스의 인스턴스를 호출하여 입력값 x의 제곱을 계산합니다.
    return f(x)

def exp(x):
    # Exp 클래스의 인스턴스를 생성합니다.
    f = Exp()
    
    # 생성된 Exp 클래스의 인스턴스를 호출하여 입력값 x의 지수를 계산합니다.
    return f(x)

# 한 줄로 작성
"""def square(x):
    return Square()(x) 

def exp(x):
    return Exp()(x)"""

'def square(x):\n    return Square()(x) \n\ndef exp(x):\n    return Exp()(x)'

In [21]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [22]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [23]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        
        # Variable 클래스의 grad 속성이 None일 경우, 해당 변수의 미분값을 1로 설정
        if self.grad is None:
            # y.grad = np.array(1.0)을 대신함
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()   # 함수를 가져온다.
            x, y = f.input, f.output   # 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)   # backward 메서드를 호출한다.
            
            if x.creator is not None:
                funcs.append(x.creator) # 얻은 입력 변수로 하나 앞의 함수를 리스트에 추가한다.

In [24]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
y.backward()
print(x.grad)

3.297442541400256


In [25]:
class Variable:
    def __init__(self, data):
        # 입력 데이터가 None이 아닌 경우, 입력 데이터의 타입이 np.ndarray인지 확인
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        # 변수의 데이터를 입력 데이터로 설정
        self.data = data
        # 변수의 기울기 초기화
        self.grad = None
        # 변수를 생성한 함수(연산) 초기화
        self.creator = None

In [26]:
x = Variable(np.array(1.0)) # 올바른 예
x = Variable(None) # 올바른 예

# x = Variable(1.0) # 잘못된 예 ( 오류 발생 )

In [27]:
x = np.array([1.0]) #  NumPy 배열 x를 생성하여 값 [1.0]으로 초기화
y = x ** 2

print(type(x), x.ndim) # x의 자료형과 차원을 출력
print(type(y)) # y의 자료형을 출력

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [28]:
x = np.array(1.0) # 0차원 배열 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y)) # np.float64

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [29]:
# 주어진 입력을 NumPy 배열로 변환하는 함수
def as_array(x):
    if np.isscalar(x):  # 입력이 스칼라인지 확인
        return np.array(x)  # 스칼라인 경우, 배열로 변환하여 반환
    return x  # 스칼라가 아닌 경우, 그대로 반환

In [30]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # forward 메서드에서 정의된 대로 계산 수행
        # as_array 함수를 사용하여 NumPy 배열로 변환
        output = Variable(as_array(y))
        # 출력값이 함수에 의해 계산되었다는 정보를 저장
        output.set_creator(self)
        self.input = input
        self.output = output
        return output